# Spring 2021 Livox lidar data processing

Script to process raw .bin files from the livo lidar collected from March 2021 to June 2021
S. Filhol 2021

-------
Part 0: Convert all point cloud to daily netcdf
For each day:
1. convert to LAS
2. rotate LAS
3. extract Geotiff
4. compile daily geotiff to netcdf

-------
Part 1:  Find out when to convert bin file into DEMs

1. get wind, temperature, flux, precip and snow depth from flux station
2. identify events: precip, wind, transport.
3. create list of timestamp to convert bin to netcdf. High temporal resolution during events, low temporal resolution in low activity

-------
Part 2

1. write code to load files metadata to sort which file to process and post process
2. 


In [63]:
#from wsn_client import query
import xarray as xr
import glob
import pandas as pd
import openpylivox as opl
import pdal, json, os

import configparser, logging, sys
sys.path.append('/home/arcticsnow/github/SnowyOwl/')
from appV2 import process_pcl as pp

In [64]:
# 1. Convert all point clouds to netcdf4 via geotiff

dir_bin = '/media/arcticsnow/My Book/SO_spring_2021_processing/bin/'
dir_netcdf = '/media/arcticsnow/My Book/SO_spring_2021_processing/netcdf/'
dir_daily = '/media/arcticsnow/My Book/SO_spring_2021_processing/daily/'

flist = glob.glob(dir_bin + '*.bin')
flist.sort()

In [65]:
meta = pd.DataFrame({'fname_archive':flist})

In [66]:
# create dataframe of file metadata
meta = pd.DataFrame({'fname_archive':flist})
#extract timestamp from filename
meta['tst'] = pd.to_datetime(meta.fname_archive.apply(lambda x: x.split('/')[-1][:-4]), format="%Y.%m.%dT%H-%M-%S")
meta['daily']=(meta.tst - pd.to_datetime('2021-03-11 00:00:00')).dt.days

In [67]:
meta.tst.diff().median()

Timedelta('0 days 00:03:00')

In [71]:
# Script to convert geotiff to netcdf
import datetime
from osgeo import gdal

def fillnodata(fname, band=1, maxSearchDist=5, smoothingIterations=0):
    ET = gdal.Open(fname, gdal.GA_Update)
    ETband = ET.GetRasterBand(band)
    result = gdal.FillNodata(targetBand=ETband, maskBand=None,
                             maxSearchDist=maxSearchDist, smoothingIterations=smoothingIterations)
    ETband = None
    ET = None
    

def raster_to_ds_daily(dir_input, dir_output, compression=True, filename_format='%Y%m%d.nc'):
    """
    function to store geotif into daily netcdf

    :param dir_input: path to folder with geotif, str
    :param dir_output: path to output folder, str
    :param compression: copmress netcdf or not, defaults to True,  bool, optional
    :param filename_format: output file name format following datetime system, defaults to '%Y%m%d.nc',  str, optional
    """    

    # list filename
    flist = glob.glob(dir_input + '*.tif')
    flist.sort()
    print(flist)
    for f_rast in flist:
        fillnodata(f_rast)
        
    # create dataframe of file metadata
    meta = pd.DataFrame({'fname':flist})
    #extract timestamp from filename
    meta['tst'] = pd.to_datetime(meta.fname.apply(lambda x: x.split('/')[-1][:-4]))

    # Create on netcdf file per day
    for date in meta.tst.dt.day.unique():
        # create time variable
        time_var = xr.Variable('time', meta.tst.loc[meta.tst.dt.day==date])
        # open raster files in datarray
        geotiffs_da = xr.concat([xr.open_rasterio(i) for i in meta.fname.loc[meta.tst.dt.day==date]], dim=time_var)
        # drop all NaN values
        geotiffs_da = geotiffs_da.where(geotiffs_da!=-9999, drop=True)
        # rename variables with raster band names
        var_name = dict(zip(geotiffs_da.band.values, geotiffs_da.descriptions))
        geotiffs_ds = geotiffs_da.to_dataset('band')
        geotiffs_ds = geotiffs_ds.rename(var_name)

        # save to netcdf file
        fname_nc = dir_output + meta.tst.loc[meta.tst.dt.day==date].iloc[0].strftime(filename_format)
        if compression:
            encode = {"min":{"compression": "gzip", "compression_opts": 9}, 
                        "max":{"compression": "gzip", "compression_opts": 9},
                        "mean":{"compression": "gzip", "compression_opts": 9},
                        "idw":{"compression": "gzip", "compression_opts": 9},
                        "count":{"compression": "gzip", "compression_opts": 9},
                        "stdev":{"compression": "gzip", "compression_opts": 9}}
            geotiffs_ds.to_netcdf(fname_nc,  encoding=encode, engine='h5netcdf')
        else:
            geotiffs_ds.to_netcdf(fname_nc)
        print('File saved: ', fname_nc)

        # clear memory cache before next loop
        geotiffs_da = None
        geotiffs_ds = None
    for file in flist:
        os.remove(file)


In [74]:
# Copy bin file for the day
import shutil
from appV2 import geotiff2netcdf as gn
import process as pp
sampling_interval=180


for day in meta.daily.unique()[4:6]:
    print('......')
    print('Processing Day ', day)
    daily = meta.loc[meta.daily==day]
    for file in daily.fname_archive:
        try:
            tst_data = pd.to_datetime(file.split('/')[-1][:19],format="%Y.%m.%dT%H-%M-%S")
            if (tst_data.second + 60*tst_data.minute + 3600*tst_data.hour) % sampling_interval == 0:
                shutil.copy(file, dir_daily)
        except IOerror:
            print('ERROR: cannot move ', file)
    print('-- File moved')
    pp.convert_bin_to_las(path_to_data=dir_daily)
    print('-- Bin converted to las')
    pp.tmp_rotate_point_clouds(z_range='[-20:20]', crop_corners='([-20, 10], [-5, 5])')
    print('-- Rotated pcl')
    pp.extract_dem(GSD= 0.02,
                origin_x=-16.2,
                origin_y=-4,
                height=7.8,
                width=22.3,
                sampling_interval=180,
                method='pdal',
                path_to_data=dir_daily,
                delete_las=True,
                tif_to_zip=False)
    print('-- Tif extracted')
    raster_to_ds_daily(dir_daily, dir_netcdf)
    print('-- daily Netcdf compiled')
    

......
Processing Day  21


   :   0%|          | 0/465897 [00:00<?, ? pts/s]

-- File moved

CONVERTING OPL BINARY DATA, PLEASE WAIT...


   : 100%|██████████| 465897/465897 [00:08<00:00, 57033.41 pts/s] 


   - Point data was converted successfully to LAS, see file: /media/arcticsnow/My Book/SO_spring_2021_processing/daily/2021.04.01T23-57-00.bin.las
     * OPL point data binary file has been deleted



FileNotFoundError: [Errno 2] No such file or directory: '/media/arcticsnow/My Book/SO_spring_2021_processing/daily/2021.04.01T23-57-00.bin'

In [58]:
meta.shape


(25388, 3)

In [ ]:
# Create on netcdf file per day
for day in meta.daily.unique():
    # create time variable
    time_var = xr.Variable('time', meta.tst.loc[meta.tst.dt.day==date])

    # convert by daily batches. Use functions from process_pcl!!!!
    pp.convert_bin_to_las()
    pp.rotate_point_clouds()
    pp.extract_dem()